In [1]:
# import liabries
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
#  src url and initial soup
# get url list for data scraping
url = 'https://www.who.int/csr/sars/country/en/'
bs0 = bs(requests.get(url).content,'html')
bs1 = bs0.find_all('a')
lst = list(map(lambda x: x if '/csr/sars/country/' in str(x) else 0, bs1))
who_main = 'http://www.who.int'
url_lst = list(map(lambda x: who_main+x,[lst[i]['href'] for i in range(len(lst)) if lst[i]!=0] ))

# url list to use for scraping
url_use = [i for i in url_lst if len(bs(requests.get(i).content,'html').find_all('tr'))!=0]

In [3]:
# normalize and retrieve list of date for df date column 
data_date = list(map(lambda x:re.findall('\d+_\d+_\d+',x)[0],url_use[3:]))
date_lst = ['2003_03_17','2003_03_18','2003_03_19']

#df date col data
data_date = date_lst+data_date


In [4]:
# define function to scrap data using url

def get_data(url,date):
    try:
        soup_table = bs(requests.get(url).content,'html').find_all('tr')
        rows = []
        for i in range(len(soup_table)):
            if '/b' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('b')])
            elif '/p' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('p')])
            elif '/td' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('td')]) 
        
        df = pd.DataFrame(data = rows[1:], columns=rows[0])
        df['Date'] = date
        result = [rows, df['Date']]
        return result
                # df.to_csv('/c/Users/sfy/Documents/Ironhack/Personal_Projects/data/%s')
    except:
        print('ValueError! Check source.',url)
        # raise ValueError('Check Column List Length')
        


In [132]:
# define function to clean data
def clean_col(x):
    cln_lst = x
    if type(cln_lst) == list:
        if len(cln_lst) != 0:
            for i in range(len(cln_lst[0])):
                if 'transmission' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Local_Chain(s)_of_Transmission'
                elif 'Country' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Country'
                elif ('Cumulative number' in cln_lst[0][i]) & ('Date for which cumulative number of cases is current' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Cumulative_Number_Of_Case(s)'
                elif ('Number of deaths' in cln_lst[0][i]) & ('Final Status/ Number of deaths' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Of_Deaths'
                elif ('Number recovered' in cln_lst[0][i]) & ('Final Status/ Number recovered' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Recovered'

    return cln_lst

def cln_trans(x):
    if 'None' in str(x):
        x = 'N'
    elif 'Yes' in str(x):
        x = 'Y'
    else:
        x = 'To be determined'
    return x

def cln_num_col(x):
    x = str(x).replace(' ','').replace('§','').replace('*','')
    if x == '':
        x = '0'
    return x

def cln_Country(x):
    x = x.lstrip().rstrip().split('^')[0]
    return x

# # define function to get df
def get_df(data,date):
    df = pd.DataFrame(data=data[1:],columns=data[0])
    df['Date'] = date
    return df



In [6]:
origin = list(map(get_data,url_use,data_date))

ValueError! Check source. http://www.who.int/csr/sars/country/table/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_10/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_11/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_12/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_14/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_15/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_16/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_17/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_18/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_19/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_21/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_22/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_23/en/
Valu

In [7]:
data_origin = [origin[i][0] for i in range(len(origin)) if type(origin[i])==list]
data_df = list(map(clean_col,data_origin))

In [221]:
date_df = [origin[i][1] for i in range(len(origin)) if type(origin[i])==list]
lista = list(map(get_df, data_df, date_df))

In [332]:
dfa = pd.concat(lista)
dfb = dfa[dfa.columns[:5]]
dfb['Cumulative_Number_Of_Case(s)'] = list(map(cln_num_col,dfb['Cumulative_Number_Of_Case(s)']))
dfb['Number_Of_Deaths'] = list(map(cln_num_col,dfb['Number_Of_Deaths']))
dfb['Local_Chain(s)_of_Transmission'] = list(map(cln_trans,dfb['Local_Chain(s)_of_Transmission']))
dfb['Cumulative_Number_Of_Case(s)']=dfb['Cumulative_Number_Of_Case(s)'].astype(int)
dfb['Number_Of_Deaths']=pd.Series(list(map(lambda x: '0' if x =='nan' else x, dfb['Number_Of_Deaths']))).astype(int)
dfb['Country'] = list(map(cln_Country,dfb['Country']))
dfb['Country'] = list(map(lambda x: 'China' if 'China' in x else x, dfb['Country']))
dfb.reset_index(inplace=True)
dfb.drop(columns='index',inplace=True)
dfb.drop(dfb.index[[1585,1586]],inplace=True)
dfb

,Country,Cumulative_Number_Of_Case(s),Number_Of_Deaths,Local_Chain(s)_of_Transmission,Date
0,Germany,2,0,N,2003_03_18
1,Canada,8,2,Y,2003_03_18
2,China,0,0,To be determined,2003_03_18
3,Singapore,23,0,Y,2003_03_18
4,China,123,1,Y,2003_03_18
5,China,3,0,Y,2003_03_18
6,Thailand,1,0,N,2003_03_18
7,Viet Nam,57,1,Y,2003_03_18
8,Slovenia,1,0,N,2003_03_18
9,United Kingdom,1,0,N,2003_03_18


In [339]:
dfc=dfb
totalindex = dfc[dfc.Country=='Total'].index
dfc.drop(totalindex , inplace=True)
dfc # without total num
# totalindex = dfb[dfb.Country=='Total'].index
# totalindex

,Country,Cumulative_Number_Of_Case(s),Number_Of_Deaths,Local_Chain(s)_of_Transmission,Date
0,Germany,2,0,N,2003_03_18
1,Canada,8,2,Y,2003_03_18
2,China,0,0,To be determined,2003_03_18
3,Singapore,23,0,Y,2003_03_18
4,China,123,1,Y,2003_03_18
5,China,3,0,Y,2003_03_18
6,Thailand,1,0,N,2003_03_18
7,Viet Nam,57,1,Y,2003_03_18
8,Slovenia,1,0,N,2003_03_18
9,United Kingdom,1,0,N,2003_03_18


In [345]:
dfc['Local_Chain(s)_of_Transmission'].value_counts() # column to be dropped

To be determined    1813
N                    318
Y                    164
Name: Local_Chain(s)_of_Transmission, dtype: int64

In [350]:
ts = dfc.groupby(['Date','Country']).agg('sum')
ts

4
           India                                           3                 2
           Indonesia                                       2                 0
           Italy                                           5                 0
           Kuwait                                          1                 5
           Malaysia                                        5                 0
           Mongolia                                        9                 0
           New Zealand                                     1                 0
           Philippines                                    14                 0
           Republic of Ireland                             1                 0
           Republic of Korea                               3                 0
           Romania                                         1                 0
           Russian Federation                              1                 2
           Singapore                                     206                 9
           South Africa                                    1                 2
           Spain                                           1                 0
           Sweden                                          3                 0
           Switzerland                                     1                 6
           Thailand                                        9                 0
           United Kingdom                                  4                 0
           United States                                  73                 0
           Viet Nam                                       63                 0
2003_07_02 Australia                                       5                 0
           Brazil                                          1                 2
           Canada                                        252                 0
           China                                        7759                 1
           Colombia                                        1                 1
           Finland                                         1                 0
           France                                          7                 0
           Germany                                        10                 4
           India                                           3                 2
           Indonesia                                       2                 0
           Italy                                           4                 0
           Kuwait                                          1                 5
           Malaysia                                        5                 0
           Mongolia                                        9                 0
           New Zealand                                     1                 0
           Philippines                                    14                 0
           Republic of Ireland                             1                 0
           Republic of Korea                               3                 0
           Romania                                         1                 0
           Russian Federation                              1                 2
           Singapore                                     206                 9
           South Africa                                    1                 2
           Spain                                           1                 0
           Sweden                                          3                 0
           Switzerland                                     1                 6
           Thailand                                        9                 0
           United Kingdom                                  4                 0
           United States                                  73                 0
           Viet Nam                                       63                 0
2003_07_03 Australia                            

In [351]:
date = []
country = []
positive = []
death = []
for i in range(len(ts.index)):
    date.append(ts.index[i][0])
    country.append(ts.index[i][1])
    positive.append(ts.values[i][0])
    death.append(ts.values[i][1])
data = [date,country,positive,death]

In [352]:
new_df = pd.DataFrame(data=data,index=['Date','Country', 'Cumulative_Number_Of_Case(s)', 'Number_Of_Deaths']).T
new_df.groupby('Date').agg('sum')

,Country,Cumulative_Number_Of_Case(s),Number_Of_Deaths
Date,,,
2003_03_18,CanadaChinaGermanySingaporeSloveniaThailandUni...,219,4
2003_03_19,CanadaChinaGermanySingaporeSloveniaSpainThaila...,264,10
2003_03_20,CanadaChinaGermanySingaporeSloveniaSpainSwitze...,306,10
2003_03_21,CanadaChinaGermanyItalyRepublic of IrelandSing...,350,15
2003_03_22,CanadaChinaGermanyItalyRepublic of IrelandSing...,386,15
2003_03_24,CanadaChinaFranceGermanyItalyRepublic of Irela...,456,15
2003_03_25,CanadaChinaFranceGermanyItalyRepublic of Irela...,487,10
2003_03_26,CanadaChinaFranceGermanyItalyRepublic of Irela...,1323,10
2003_03_27,CanadaChinaFranceGermanyItalyRepublic of Irela...,1408,15
